### Forest structure using PDAL + Python

Dr Adam Steer, November 2019.

In [1]:
NODATA_VALUE = -9999
LCF_HEIGHTS = [0, 0.05, 0.5, 1, 2, 3]


In [2]:
#imports
import pdal
import numpy as np
import json

from shapely.geometry import Point
from shapely.geometry import MultiPolygon
from shapely.geometry import box
#from shapely.strtree import STRtree

import geopandas as gpd
import pandas as pd
import osmnx as ox

import os

import sys

sys.path.insert(0, os.path.abspath('../'))
sys.path.insert(0, os.path.abspath('../../callingelvis'))

# not using this, using geopandas instead
from rtree import index

# this is needed to create a raster from the output array
from osgeo import gdal
import osgeo.osr as osr

In [3]:
from callingelvis import anybodyhome

import forestmetrics.forestmetrics as metrics
import forestmetrics.utils as forestutils

#import forestpipeline


In [4]:
def compute_tern_products(metadata, points, sindex, resolution, lasfile, outpath):
    """
    Wrapper to iterate over the input data and generate rasters for each product.
    
    *note this part could be paralellised - maybe per-product, or per-cell
    
    Each grid square processed in this loop corresponds to one pixel in an output raster.
    
    """
    
    #set up an 'output resolution' sized grid - like a fishnet grid.
    # each polygon in the resulting set covers an area of 'resolution X resolution'
    pixel_grid = forestutils.gen_raster_cells(metadata, resolution)
    
    #set up output rasters
    
    # get tile width and height
    tile_width = metadata["metadata"]["readers.las"]["maxx"] - metadata["metadata"]["readers.las"]["minx"]
    tile_height = metadata["metadata"]["readers.las"]["maxy"] - metadata["metadata"]["readers.las"]["miny"]

    raster_xsize = int(np.ceil(tile_width) / resolution)
    raster_ysize = int(np.ceil(tile_height) / resolution)
    
    #replicate for all products...
    vh_raster = np.zeros((raster_xsize, raster_ysize))
    vcf_raster = np.zeros((raster_xsize, raster_ysize))
    cth_raster = np.zeros((raster_xsize, raster_ysize))
    cbh_raster = np.zeros((raster_xsize, raster_ysize))
    fbf_raster = np.zeros((raster_xsize, raster_ysize))
    cli_raster = np.zeros((raster_xsize, raster_ysize))
    density_raster = np.zeros((raster_xsize, raster_ysize))
    
    veg_below_dict = {}

    veg_below_dict["all"] = np.zeros((raster_xsize, raster_ysize))
    for height in LCF_HEIGHTS:
        veg_below_dict[str(height)] = np.zeros((raster_xsize, raster_ysize))
    
    #internal loop around grid squares covering the LAS tile.
    # this is another ppoint for parallelisation - since we can set up a list of geometries
    # and cast that at multipuple processes, setting up one process per grid square
    # another way to do this would be to recast this loop block into a function which can  
    # be called by one process per product
    # the second strategy seems easier, then only one process is trying to write into each
    # output array.
    
    for pixel in pixel_grid:
        
        #compute output array index for this cell:
        poly_x, poly_y = pixel.centroid.xy
        
        poly_base_x = poly_x[0] - metadata["metadata"]["readers.las"]["minx"]
        poly_base_y = poly_y[0] - metadata["metadata"]["readers.las"]["miny"]
        
        array_x = int(np.floor((poly_base_x / (resolution)) ))
        array_y = int(np.floor((poly_base_y / (resolution)) ))
                
        #get points for this cell
        matches = forestutils.get_cell_points(pixel, points, sindex)
        
        #compute in order
        #VH
        vh_raster[array_x, array_y] = metrics.comp_vh(matches)
        
        #VCF
        vcf_raster[array_x, array_y] = metrics.comp_vcf(matches)
        
        #LCF - long-ish process..
        # compute a dictionary of points below height thresholds
        veg_below = metrics.comp_veg_layers(matches, LCF_HEIGHTS)
        
        # add the first element of the dictionary to a raster output
        veg_below_dict["all"][array_x, array_y] = veg_below["all"]
        
        #iterate over the height thresholds and do likewise...
        for height in LCF_HEIGHTS:
            veg_below_dict[str(height)][array_x, array_y] = veg_below[str(height)]
        
        #CTH
        cth_raster[array_x, array_y] = metrics.comp_cth(matches)
        
        #CBH
        cbh_raster[array_x, array_y] = metrics.comp_cbh(matches)
        
        #FBF
        fbf_raster[array_x, array_y] = metrics.comp_fbf(matches)
        
        #CLI
        cli_raster[array_x, array_y] = metrics.comp_cli(matches)
        
        #density
        density_raster[array_x, array_y] = metrics.comp_density(matches, resolution)


    #end of computing stuff, time to make outputs...
    
    #compute LCF values given our height thresholded veg counts
    lcf = metrics.comp_lcf(veg_below_dict, vcf_raster)
    
    if (not os.path.isdir(outpath + "/dem")):
        os.mkdir(outpath + "/dem")
    
    dem = forestutils.comp_dem(lasfile, outpath, resolution)
    
    tern_products = {}
    tern_products["vh"] = vh_raster
    tern_products["vcf"] = vcf_raster
    tern_products["cth"] = cth_raster
    tern_products["cbh"] = cbh_raster
    tern_products["fbf"] = fbf_raster
    tern_products["cli"] = cli_raster
    tern_products["lcf_h"] = lcf["lcf_h"]
    tern_products["lcf_os"] = lcf["lcf_os"]
    tern_products["lcf_us"] = lcf["lcf_us"]
    tern_products["lcf_cf"] = lcf["lcf_cf"]
    tern_products["lcf_ef"] = lcf["lcf_ef"]
    tern_products["lcf_nsf"] = lcf["lcf_nsf"]
    tern_products["density"] = density_raster

    return(tern_products)

In [5]:
def export_tern_products(tern_products, metadata, resolution, lasfile, outpath):
    
    #set up GDAL parameters
    
    wktcrs = metadata["metadata"]["readers.las"]["comp_spatialreference"]
    
    raster_parameters = {}
    raster_parameters["width"] = np.shape(tern_products["vh"])[0]
    raster_parameters["height"] = np.shape(tern_products["vh"])[1]
    raster_parameters["upperleft_x"] = metadata["metadata"]["readers.las"]["minx"]
    raster_parameters["upperleft_y"] = metadata["metadata"]["readers.las"]["maxy"]
    raster_parameters["resolution"] = resolution
    raster_parameters["projection"] = wktcrs
    
    fileroot = forestutils.make_file_rootname(lasfile)
    print(fileroot)
    
    for productname in tern_products.keys():
    
        if (not os.path.isdir(os.path.join(outpath,
                                       productname))):
            os.mkdir(os.path.join(outpath,
                                  productname))
    
    
        #set output filenames
        separator = "-"
        
        raster_name = separator.join([fileroot,
                                     productname,
                                     str(resolution) + "m.tiff"])
        raster_path = os.path.join(outpath,
                                   productname,
                                   raster_name)
        print(raster_path)
        forestutils.write_product_geotiff(tern_products[productname], raster_path, raster_parameters)


    return()

## Testing functionality using a local file
The following section generates metrics from a local LAZ file. Plugging in download mechanics from ELVIS will be added later

In [6]:
#lidar test file - Mt Ainslie, chosen for varied vegetation cover and topography
# this is pretty big, try it out if you've got more resources than my macbook pro!

# thinking ahead, there will probably end up being a file splitting pre-process for 
# tiles like these... capping at say, 20 mill points. Sorting data before splitting
# will be essential.

#lasfile = "/Volumes/Antares/ACT-lidar/8ppm/callingelvis-testdata/ACT2015_8ppm-C3-AHD_6966094_55.laz"

In [7]:
#lasfile = "/Volumes/Antares/fire-test/NSW Government - Spatial Services-2/Point Clouds/AHD/StAlbans201709-LID2-C3-AHD_2866308_56_0002_0002/StAlbans201709-LID2-C3-AHD_2866308_56_0002_0002.las"

In [8]:
lasfile = "../../callingelvis-sampledata/Berridale201802-LID2-C3-AHD_6585974_55_0002_0002.las"

In [9]:
lasfile = "../../callingelvis-sampledata/uncompahgre.laz"

In [10]:
# dump everything from memory
points = None
df = None
tern_products = None

In [11]:
%%time

metadata = forestutils.readlasmetadata(lasfile)

CPU times: user 37.9 ms, sys: 9.81 ms, total: 47.7 ms
Wall time: 67.4 ms


In [12]:
%%time
# this part of the process is simply reading from the source file into a Numpy 
# array. No analysis yet.

points = forestutils.readlasfile(lasfile)

CPU times: user 1min 8s, sys: 868 ms, total: 1min 9s
Wall time: 1min 9s


In [13]:
# uncomment to examine LAS metadata
#metadata

In [14]:
#points = [ points[0]["X"], points[0]["Y"], points[0]["Z"]]

#,"Y","Z","ReturnNumber","NumberofReturns","HeightAboveGround"]

In [15]:
points

[array([(245348.175, 4208778.206, 3203.949, 8355, 1, 1, 0, 1, 4, 29., 0, 0, 315234.53773385, -83.259),
        (245352.246, 4208775.81 , 3203.993, 5734, 1, 1, 0, 1, 4, 29., 0, 0, 315234.5605584 , -83.215),
        (245356.252, 4208773.495, 3204.325, 3604, 1, 1, 0, 1, 4, 29., 0, 0, 315234.58338235, -82.883),
        ...,
        (239353.378, 4228497.921, 2740.002, 1638, 1, 1, 0, 0, 4, 14., 0, 0, 315058.92309751,  -7.239),
        (239364.602, 4228499.343, 2740.108, 2129, 1, 1, 0, 0, 4, 14., 0, 0, 315058.92313691,  -7.261),
        (239367.467, 4228499.71 , 2740.228, 1474, 1, 1, 0, 0, 4, 14., 0, 0, 315058.92314691,  -7.141)],
       dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'), ('ReturnNumber', 'u1'), ('NumberOfReturns', 'u1'), ('ScanDirectionFlag', 'u1'), ('EdgeOfFlightLine', 'u1'), ('Classification', 'u1'), ('ScanAngleRank', '<f4'), ('UserData', 'u1'), ('PointSourceId', '<u2'), ('GpsTime', '<f8'), ('HeightAboveGround', '<f8')])]

In [16]:
%%time

#here we read points into a GeoDataFrame and dump the labelled array.
# this is a pretty expensive step RAM wise, we're duplicating all the points...
#df = gpd.GeoDataFrame(points)
# ...and we're adding a geometry column... probably slows thinsg down a bit...

df = forestutils.pdal2df(points)

# set the points structured array to None, it isn't used anymore
#points = None

CPU times: user 6min 56s, sys: 3.14 s, total: 6min 59s
Wall time: 6min 59s


In [17]:
df

,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,UserData,PointSourceId,GpsTime,HeightAboveGround,geometry
0,245348.175,4208778.206,3203.949,8355,1,1,0,1,4,29.0,0,0,315234.537734,-83.259,POINT (245348.175 4208778.206)
1,245352.246,4208775.810,3203.993,5734,1,1,0,1,4,29.0,0,0,315234.560558,-83.215,POINT (245352.246 4208775.810)
2,245356.252,4208773.495,3204.325,3604,1,1,0,1,4,29.0,0,0,315234.583382,-82.883,POINT (245356.252 4208773.495)
3,245355.871,4208769.974,3202.111,4095,1,1,0,1,4,29.0,0,0,315234.606197,-85.097,POINT (245355.871 4208769.974)
4,245359.934,4208770.864,3203.994,4423,1,1,0,0,4,29.0,0,0,315234.606206,-83.214,POINT (245359.934 4208770.864)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13583763,239363.217,4228504.871,2739.351,1802,1,1,0,0,4,14.0,0,0,315058.877469,-8.018,POINT (239363.217 4228504.871)
13583764,239346.286,4228502.725,2739.171,2293,1,1,0,0,4,14.0,0,0,315058.877409,-8.070,POINT (239346.286 4228502.725)
13583765,239353.378,4228497.921,2740.002,1638,1,1,0,0,4,14.0,0,0,315058.923098,-7.239,POINT (239353.378 4228497.921)
13583766,239364.602,4228499.343,2740.108,2129,1,1,0,0,4,14.0,0,0,315058.923137,-7.261,POINT (239364.602 4228499.343)


In [18]:
%%time

# here we generate an RTree index on the dataframe using GeoPandas.
# also pretty expensive... 

sindex = forestutils.spatialindex(df)

CPU times: user 11min 57s, sys: 6.93 s, total: 12min 4s
Wall time: 41min 17s


In [19]:
# set an output resolution

resolution = 25

In [20]:
metadata["metadata"]["readers.las"]["minx"]

238090.886

In [21]:
points[0]["X"][1:10]

array([245352.246, 245356.252, 245355.871, 245359.934, 245359.522,
       245363.447, 245362.566, 245373.96 , 245375.131])

In [22]:
#points2 = [points[0]["X"],points[0]["Y"],points[0]["Z"],points[0]["ReturnNumber"],points[0]["NumberOfReturns"]]

In [23]:
%%time 
#df2 = gpd.GeoDataFrame(points2)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.87 µs


In [24]:
df2 = None

In [25]:
points2 = None 

In [26]:
points = None

In [ ]:
%%time

#produce some rasters!

tern_products = compute_tern_products(metadata, df, sindex, resolution, lasfile, "../..")

In [ ]:
#write some rasters

export_tern_products(tern_products, metadata, resolution, lasfile, "../../")

In [ ]:
resolution = 5

In [ ]:
%%time

#produce some rasters!

tern_products = compute_tern_products(metadata, df, sindex, resolution, lasfile, "../..")

In [ ]:
from matplotlib import pyplot as plt

%matplotlib inline

In [ ]:
tern_products.keys()

In [ ]:
plt.imshow(np.rot90(tern_products["vcf"]))
plt.colorbar()
plt.title("Vegetation cover fraction (VCF)")

In [ ]:
plt.imshow(np.rot90(tern_products["cth"]), vmin = 0)
plt.colorbar()
plt.title("Canopy top height (CTH)")


In [ ]:
plt.imshow(np.rot90(tern_products["cbh"]), vmin = 0)
plt.colorbar()
plt.title("Canopy base height (CBH)")

In [ ]:
plt.imshow(np.rot90(tern_products["cth"] -tern_products["cbh"]))
plt.colorbar()
plt.title("Canopy top and base height difference")

In [ ]:
plt.imshow(np.rot90(tern_products["fbf"]))
plt.colorbar()
plt.title("Building fraction (FBF)")

In [ ]:
plt.imshow(np.rot90(tern_products["cli"]))
plt.colorbar()
plt.title("Canopy Layering Index (CLI)")

In [ ]:
plt.imshow(np.rot90(tern_products["lcf_h"]))
plt.colorbar()
plt.title("LCF_H")

In [ ]:
plt.imshow(np.rot90(tern_products["lcf_os"]))
plt.colorbar()
plt.title("LCF_OS")

In [ ]:
plt.imshow(np.rot90(tern_products["lcf_us"]))
plt.colorbar()
plt.title("LCF_US")

In [ ]:
plt.imshow(np.rot90(tern_products["lcf_cf"]))
plt.colorbar()
plt.title("LCF_CF")

In [ ]:
plt.imshow(np.rot90(tern_products["density"]))
plt.colorbar()
plt.title("density")

In [ ]:
polygons = forestutils.gen_raster_cells(metadata, resolution)

In [ ]:
polygons[40]

In [ ]:
matches = forestutils.get_cell_points(polygons[55], df, sindex)

In [ ]:
metrics.comp_cli(matches)

In [ ]:
metrics.comp_density(matches, resolution)

## code purgatory

In [ ]:
%%time
# an attempt at building an index without pandas. Pandas was far easier...

## rtree index building straight from the point dataset... which also duplicates the point set...
# I think this is a pretty slow way... if we're looping over points might as well process them
# at the same time...
idx = index.Index()
for pid, point in enumerate(points[0]):
    idx.insert(pid, (point[0], point[1], point[0], point[1]), point)


In [ ]:
## for raster writing later...

### set up GDAL parameters
    
    wktcrs = metadata["metadata"]["readers.las"]["comp_spatialreference"]
    
    raster_parameters = {}
    raster_parameters["width"] = np.shape(vcf_raster)[0]
    raster_parameters["height"] = np.shape(vcf_raster)[1]
    raster_parameters["upperleft_x"] = metadata["metadata"]["readers.las"]["minx"]
    raster_parameters["upperleft_y"] = metadata["metadata"]["readers.las"]["maxy"]
    raster_parameters["resolution"] = resolution
    raster_parameters["projection"] = wktcrs
    
    fileroot = forestutils.make_file_rootname(lasfile)
    print(fileroot)
    
    if (not os.path.isdir(outpath + "/vcf")):
        os.mkdir(outpath + "/vcf")
    if (not os.path.isdir(outpath + "/cth")):
        os.mkdir(outpath + "/cth")
    if (not os.path.isdir(outpath + "/cbh")):
        os.mkdir(outpath + "/cbh")
    if (not os.path.isdir(outpath + "/fbf")):
        os.mkdir(outpath + "/fbf")
    if (not os.path.isdir(outpath + "/cli")):
        os.mkdir(outpath + "/cli")
        
    #lcf will hold 3 output rasters
    if (not os.path.isdir(outpath + "/lcf")):
        os.mkdir(outpath + "/lcf")


    
    #set output filenames
    vcf_raster_path = os.path.join(outpath,
                                   "vcf",
                                   fileroot + "-VCF-" + str(resolution) + "m.tiff")
    
    print(vcf_raster_path)
    cth_raster_path = outpath + "/cth/" + fileroot + "-CTH-" + str(resolution) + "m.tiff"
    cbh_raster_path = outpath + "/cbh/" + fileroot + "-CBH-" + str(resolution) + "m.tiff"
    fbf_raster_path = outpath + "/fbf/" + fileroot + "-FBF-" + str(resolution) + "m.tiff"
    cli_raster_path = outpath + "/cli/" + fileroot + "-CLI-" + str(resolution) + "m.tiff"


    #write geotiffs and return arrays for inspection...
    
    #TO DO:
    # - density
    # - lcf products

    forestutils.write_product_geotiff(vcf_raster, vcf_raster_path, raster_parameters)
    forestutils.write_product_geotiff(cth_raster, cth_raster_path, raster_parameters)
    forestutils.write_product_geotiff(cbh_raster, cbh_raster_path, raster_parameters)
    forestutils.write_product_geotiff(fbf_raster, fbf_raster_path, raster_parameters)
    forestutils.write_product_geotiff(cli_raster, cli_raster_path, raster_parameters)
    